<a href="https://colab.research.google.com/github/NPraveenLiyanage/weatherPredictionModel/blob/main/Weather_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

impory lib

In [ ]:
import requests #this lib help us to fetch data from API
import pandas as pd #for handling and analyzing data
import numpy as np #for numerical operations
from sklearn.model_selection import train_test_split #to split data into training and testing
from sklearn.preprocessing import LabelEncoder #to convert categorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #to build a random forest classification and regression model
from sklearn.metrics import mean_squared_error #to measure the accuracy of our predictions
from datetime import datetime, timedelta #to work with dates and times
import pytz

In [ ]:
API_Key = '270a56acbb0149f9b461c746fdd7073c'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

Fetch current weather data

In [ ]:
def fetch_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_Key}&units=metric" #construct the api request url
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'wind_gust_speed': data['wind']['speed']
    }

Read historical data

In [ ]:
def read_historical_data(filename):
    df = pd.read_csv(filename) #this read csv file into dataframe
    df = df.dropna() #remove rows with missing values
    df = df.drop_duplicates() #remove duplicate rows
    return df

Prepare data for training

In [ ]:
def prepare_data(data):
    le = LabelEncoder() #create a labelEncounter instance
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    #define feature variable and target variables
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] #feature variables
    y = data['RainTomorrow'] #target variable

    return X, y, le  #return feature variable, target variable and label encoder

Train Rain predication model

In [ ]:
def train_rain_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train) #train the model

  y_pred = model.predict(X_test) #to make prediction on test set
  accuracy = np.mean(y_pred == y_test)

  print("Mean square error for Rain model")
  print(mean_squared_error(y_test, y_pred))

  print(f'Accuracy: {accuracy}')
  return model

Prepare regression data

In [ ]:
def prepare_regression_data(data, feature):
  X, y = [], [] #initialze test for feature and target value

  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1, 1)
  y = np.array(y)

  return X, y

Train regression model

In [ ]:
def train_regression_model(X, y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, y)
  return model

Predict Future

In [ ]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])

  return predictions[1:]

Weather analysis function

In [ ]:
def weather_view():
  city = input("Enter the city name: ")
  current_weather = fetch_current_weather(city)

  #load historical data
  data = read_historical_data('/content/weather.csv')

  #prepare and train the rain prediction model

  X, y, le = prepare_data(data)
  rain_model = train_rain_model(X, y)

  #map wind direction to campass points
  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
  ]

  # Corrected comparison and added a default value for next()
  compass_direction = next((point for point, start, end in compass_points if start < wind_deg < end), "Unknown")

  # Handle the case where compass_direction is "Unknown"
  if compass_direction != "Unknown" and compass_direction in le.classes_:
      compass_direction_encoded = le.transform([compass_direction])[0]
  else:
      compass_direction_encoded = -1 # Or another value indicating unknown direction

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['wind_gust_speed'], # Corrected key here
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp'],
  }

  current_df = pd.DataFrame([current_data])

  #rain prediction

  rain_prediction = rain_model.predict(current_df)[0]

 #prepare regression model for temp and humidity

  X_temp, y_temp = prepare_regression_data(data, 'Temp')
  X_hum, y_hum = prepare_regression_data(data, 'Humidity')

  temp_model = train_regression_model(X_temp, y_temp)
  hum_model = train_regression_model(X_hum, y_hum)

  #predict future temp and humidity

  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_humidity = predict_future(hum_model, current_weather['humidity'])

  #prepare time for future prediction

  timezone = pytz.timezone('Asia/Kolkata')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  # Corrected date formatting for future_times
  future_times = [(next_hour + timedelta(hours=i)).strftime("%HH:%MM") for i in range(5)]

  #Display result

  print(f"city: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Min Temperature: {current_weather['temp_min']}°C")
  print(f"Max Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")
  # Consider displaying compass direction instead of raw degrees
  print(f"Wind Gust Direction: {compass_direction}")

  print("Future Temperature Predictions:")
  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("Future Humidity Predictions:")
  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()

Enter the city name: England
Mean square error for Rain model
0.1506849315068493
Accuracy: 0.8493150684931506
city: England, US
Current Temperature: 28°C
Feels Like: 31°C
Min Temperature: 28°C
Max Temperature: 29°C
Humidity: 70%
Weather Prediction: overcast clouds
Rain Prediction: Yes
Wind Gust Direction: SW
Future Temperature Predictions:
01H:00M: 27.7°C
02H:00M: 22.8°C
03H:00M: 25.4°C
04H:00M: 24.7°C
05H:00M: 23.4°C
Future Humidity Predictions:
01H:00M: 64.8%
02H:00M: 56.2%
03H:00M: 45.2%
04H:00M: 46.3%
05H:00M: 48.2%
